In [1]:
import pandas as pd
from datetime import date
from glob import glob
import seaborn as sns
import plotly.offline as pyo
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

# with this renderer, the plots are also shown in the html file.
pio.renderers.default = "iframe_connected"

## International COVID-data

In [2]:
today = date.today()
dates = [date.strftime('%m-%d-%Y') for date in pd.date_range(start='2020-01-22', end=today)][:-1]
dates_as_date = pd.date_range(start='2020-01-22', end=today)[:-1]


# standardize column names for all entries
def rename_columns(column):
    column_map = {
        'Lat': 'Latitude',
        'Long_': 'Longitude',
        'Incidence_Rate': 'Incident_Rate'
    }
    if column in column_map:
        return column_map[column]
    return column.replace('/', '_').replace('-', '_').replace(' ', '_')

all_cases = []
for d in range(len(dates)):
    path = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(dates[d])
    data = pd.read_csv(path)
    data.rename(columns=rename_columns, inplace=True)
    data['Date'] = dates_as_date[d]
    all_cases.append(data)

df = pd.concat(all_cases)

In [3]:
# all_cases.Country_Region.unique()

# standardize country names for all entries
country_mapping = {
    'MS Zaandam|Diamond Princess|Cruise Ship': 'Others', # move cruise ships to others
    'Hong Kong.+': 'Hong Kong',
    'Iran.+': 'Iran',
    '.*Congo.*': 'Congo',
    'Mainland China': 'China',
    '.*Bahamas.*': 'The Bahamas',
    '.*Gambia.*': 'The Gambia',
    'Viet Nam': 'Vietnam',
    'Taiwan\*': 'Taiwan',
    'Cote d\'Ivoire': 'Ivory Coast',
    'Cabo Verde': 'Cape Verde',
    'Russian Federation': 'Russia',
    ' Azerbaijan': 'Azerbaijan',
    'Holy See': 'Vatican City',
    'Republic of Ireland': 'Ireland',
    'Republic of Moldova': 'Moldova',
    'Czechia': 'Czech Republic',
    'Republic of Korea|Korea, South': 'South Korea',
    'Timor-Leste': 'East Timor',
    'Macao SAR|Macau': 'Macao',
    'UK': 'United Kingdom',
    'Jersey|Guernsey': 'Channel Islands',
    'Dominican Republicn Republic|Dominica': 'Dominican Republic'
}

df['Country_Region'].replace(
    to_replace=country_mapping.keys(),
    value=country_mapping.values(),
    regex=True,
    inplace=True
)

In [8]:
df.to_csv('worldwide.csv')

In [36]:
df = pd.read_csv('worldwide.csv')

In [3]:
# group data by country
daily_updates = df.groupby(['Country_Region', 'Date']).agg(
     Confirmed = ('Confirmed','sum'),
     Deaths = ('Deaths','sum'),
 ).reset_index()

In [4]:
# get changes in data
updates_per_country = daily_updates.groupby('Country_Region')
# diff takes difference to point in group before it
daily_updates['New_Confirmed'] = updates_per_country['Confirmed'].diff().fillna(0)
daily_updates['New_Deaths'] = updates_per_country['Deaths'].diff().fillna(0)

In [5]:
# reorder columns
worldwide_pretty = daily_updates.loc[:, ['Date', 'Country_Region', 'New_Confirmed', 'Confirmed', 'New_Deaths', 'Deaths']]

In [6]:
worldwide_pretty.head(3)

,Date,Country_Region,New_Confirmed,Confirmed,New_Deaths,Deaths
0,2020-02-24,Afghanistan,0.0,1.0,0.0,0.0
1,2020-02-25,Afghanistan,0.0,1.0,0.0,0.0
2,2020-02-26,Afghanistan,0.0,1.0,0.0,0.0


In [7]:
# create df with one column per date
worldwide_all = worldwide_pretty.groupby('Date').agg(
    Confirmed = ('Confirmed', 'sum'),
    New_Confirmed = ('New_Confirmed', 'sum')
)
# create a moving average
worldwide_all['SMA_14'] = round(worldwide_all.loc[:,'New_Confirmed'].rolling(window=14).mean())
worldwide_all = worldwide_all.reset_index()

In [8]:
worldwide_all.head(3)

,Date,Confirmed,New_Confirmed,SMA_14
0,2020-01-22,557.0,0.0,NaN
1,2020-01-23,1097.0,537.0,NaN
2,2020-01-24,941.0,-158.0,NaN


In [23]:
plot = go.Figure()

plot.add_trace(
    go.Bar(
        name='Daily Meas',
        x=worldwide_all["Date"], 
        y=worldwide_all["New_Confirmed"],
        marker={
            # size': 10,
            'color': 'red',
            'opacity': 1
        },
    )
)

plot.add_trace(
    go.Scatter(
        name='14 Day MA',
        x=worldwide_all["Date"], 
        y=worldwide_all["SMA_14"],
        mode='lines',
        marker={
            'size': 1,
            'color': 'black',
            'opacity': 0.7
        },
        line_width=1.5
    )
)


plot.update_layout(
    title_x=0,
    title='New Corona Cases Worldwide',
    margin={
        'l': 25,
        'b': 25,
        't': 50,
        'r': 20
    },
    height=400,
    yaxis={
        'color': '#748B99',
        'showgrid': True,
        'showline': True,
        'gridwidth': 1,
        'gridcolor': '#B6C3CC',
        # 'rangemode': "tozero",
        'range':
            [0,
             worldwide_all.New_Confirmed.max() * 1.05],
        # 'ticksuffix': ' '
        # 'tickprefix': 'id: '
    },
    xaxis={
        'showgrid': False,
        'color': '#748B99',
        'showline': True,
        'linecolor': '#B6C3CC',
    },
    paper_bgcolor='#FFFFFF',
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    legend={
        'yanchor': 'top',
        'y': 0.99,
        'xanchor': 'right',
        'x': 0.99
    })

plot.show()

## Swiss COVID-data

In [60]:
# data import
path = "https://raw.githubusercontent.com/openZH/covid_19/master/COVID19_Fallzahlen_CH_total_v2.csv"
chData = pd.read_csv(path)

In [62]:
# only use data from 1 june on
chData = chData[chData.date > '2020-06-01']

# remove Fürstentum Lichtenstein
chData = chData[chData.abbreviation_canton_and_fl != 'FL']

# calculate new cases and deaths
chGrouped = chData.groupby('abbreviation_canton_and_fl')
chData['new_cases'] = chGrouped['ncumul_conf'].diff().fillna(0)
chData['new_deaths'] = chGrouped['ncumul_deceased'].diff().fillna(0)

# calculate total cases and deaths
chData['total_cases'] = chGrouped['new_cases'].cumsum()
chData['total_deaths'] = chGrouped['new_deaths'].cumsum()

# remove obsolete columns
chData = chData[[
    'date',
    'abbreviation_canton_and_fl',
    'new_cases',
    'total_cases',
    'new_deaths',
    'total_deaths'
]]

# rename columns
chData = chData.rename(
    {
    'abbreviation_canton_and_fl': 'canton'
    },
    axis='columns'
)

In [63]:
chData

,date,canton,new_cases,total_cases,new_deaths,total_deaths
2384,2020-06-02,BL,0.0,0.0,0.0,0.0
2385,2020-06-02,FR,0.0,0.0,0.0,0.0
2386,2020-06-02,GR,0.0,0.0,0.0,0.0
2387,2020-06-02,JU,0.0,0.0,0.0,0.0
2388,2020-06-02,NE,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
11992,2021-06-22,GE,0.0,53542.0,0.0,463.0
11993,2021-06-22,UR,0.0,3166.0,0.0,43.0
11994,2021-06-22,SH,2.0,4703.0,0.0,70.0
11995,2021-06-22,SZ,7.0,11981.0,0.0,215.0


In [64]:
# create df with one column per date
df = chData.groupby('date').agg(
    Confirmed = ('total_cases', 'sum'),
    New_Confirmed = ('new_cases', 'sum')
)
# create a moving average
df['SMA_14'] = round(df.loc[:,'New_Confirmed'].rolling(window=14).mean())
df = df.reset_index()

In [65]:
plot = go.Figure()

plot.add_trace(
    go.Bar(
        name='Daily Meas',
        x=df["date"], 
        y=df["New_Confirmed"],
        marker={
            # size': 10,
            'color': 'red',
            'opacity': 1
        },
    )
)

plot.add_trace(
    go.Scatter(
        name='14 Day MA',
        x=df["date"], 
        y=df["SMA_14"],
        mode='lines',
        marker={
            'size': 1,
            'color': 'black',
            'opacity': 0.7
        },
        line_width=1.5
    )
)


plot.update_layout(
    title_x=0,
    title='New Corona Cases Switzerland',
    margin={
        'l': 25,
        'b': 25,
        't': 50,
        'r': 20
    },
    height=400,
    yaxis={
        'color': '#748B99',
        'showgrid': True,
        'showline': True,
        'gridwidth': 1,
        'gridcolor': '#B6C3CC',
        # 'rangemode': "tozero",
        'range':
            [0,
             df.New_Confirmed.max() * 1.05],
        # 'ticksuffix': ' '
        # 'tickprefix': 'id: '
    },
    xaxis={
        'showgrid': False,
        'color': '#748B99',
        'showline': True,
        'linecolor': '#B6C3CC',
    },
    paper_bgcolor='#FFFFFF',
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    legend={
        'yanchor': 'top',
        'y': 0.99,
        'xanchor': 'right',
        'x': 0.99
    })

plot.show()

In [70]:
# create df with one column per date
df = chData[chData.canton == 'BE']
# create a moving average
df['SMA_14'] = round(df.loc[:,'new_cases'].rolling(window=14).mean())
df = df.reset_index()

<ipython-input-70-5ba604c9944e>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [71]:
plot = go.Figure()

plot.add_trace(
    go.Bar(
        name='Daily Meas',
        x=df["date"], 
        y=df["new_cases"],
        marker={
            # size': 10,
            'color': 'red',
            'opacity': 1
        },
    )
)

plot.add_trace(
    go.Scatter(
        name='14 Day MA',
        x=df["date"], 
        y=df["SMA_14"],
        mode='lines',
        marker={
            'size': 1,
            'color': 'black',
            'opacity': 0.7
        },
        line_width=1.5
    )
)


plot.update_layout(
    title_x=0,
    title='New Corona Cases Bern',
    margin={
        'l': 25,
        'b': 25,
        't': 50,
        'r': 20
    },
    height=400,
    yaxis={
        'color': '#748B99',
        'showgrid': True,
        'showline': True,
        'gridwidth': 1,
        'gridcolor': '#B6C3CC',
        # 'rangemode': "tozero",
        'range':
            [0,
             df.new_cases.max() * 1.05],
        # 'ticksuffix': ' '
        # 'tickprefix': 'id: '
    },
    xaxis={
        'showgrid': False,
        'color': '#748B99',
        'showline': True,
        'linecolor': '#B6C3CC',
    },
    paper_bgcolor='#FFFFFF',
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False,
    legend={
        'yanchor': 'top',
        'y': 0.99,
        'xanchor': 'right',
        'x': 0.99
    })

plot.show()

In [45]:
chData

,date,canton,total_cases,total_deaths,new_cases,new_deaths
2363,2020-06-01,BL,1.0,0.0,0.0,0.0
2364,2020-06-01,FR,0.0,0.0,0.0,0.0
2365,2020-06-01,GR,0.0,0.0,0.0,0.0
2366,2020-06-01,JU,0.0,0.0,0.0,0.0
2367,2020-06-01,NE,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
11988,2021-06-21,ZH,108922.0,1281.0,11.0,0.0
11990,2021-06-22,GE,53546.0,463.0,0.0,0.0
11991,2021-06-22,UR,3169.0,43.0,0.0,0.0
11992,2021-06-22,SH,4718.0,72.0,2.0,0.0
